# Network Robustness Analysis

This notebook demonstrates network robustness analysis and resilience testing.

## Learning Objectives
- Understand network robustness measures
- Analyze failure scenarios
- Study percolation theory
- Implement attack strategies

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Random Failures

Analyze network behavior under random node failures.

In [ ]:
def analyze_random_failures(G, max_removal_fraction=0.8, n_trials=10):
    """Analyze network robustness under random node failures"""
    results = []
    n_nodes = G.number_of_nodes()
    
    for removal_fraction in np.linspace(0, max_removal_fraction, 20):
        n_remove = int(removal_fraction * n_nodes)
        giant_component_sizes = []
        
        for _ in range(n_trials):
            # Randomly remove nodes
            nodes_to_remove = np.random.choice(list(G.nodes()), n_remove, replace=False)
            G_temp = G.copy()
            G_temp.remove_nodes_from(nodes_to_remove)
            
            # Find largest component
            if G_temp.number_of_nodes() > 0:
                largest_cc = max(nx.connected_components(G_temp), key=len)
                giant_component_sizes.append(len(largest_cc) / n_nodes)
            else:
                giant_component_sizes.append(0)
        
        results.append({
            'removal_fraction': removal_fraction,
            'mean_giant_component': np.mean(giant_component_sizes),
            'std_giant_component': np.std(giant_component_sizes)
        })
    
    return pd.DataFrame(results)

# Test on a scale-free network
G_scale_free = nx.barabasi_albert_graph(100, 3)
results = analyze_random_failures(G_scale_free)

print("Random Failure Analysis Results:")
print(results.head())

In [ ]:
# Plot results
plt.figure(figsize=(10, 6))
plt.errorbar(results['removal_fraction'], results['mean_giant_component'], 
            yerr=results['std_giant_component'], fmt='o-', capsize=5)
plt.xlabel('Fraction of Nodes Removed')
plt.ylabel('Giant Component Size (Normalized)')
plt.title('Network Robustness: Random Failures')
plt.grid(True, alpha=0.3)
plt.show()

## 2. Targeted Attacks

Analyze network behavior under targeted attacks based on centrality measures.

In [ ]:
def analyze_targeted_attacks(G, centrality_func, max_removal_fraction=0.8):
    """Analyze network robustness under targeted attacks"""
    results = []
    n_nodes = G.number_of_nodes()
    
    # Calculate centrality
    centrality = centrality_func(G)
    
    for removal_fraction in np.linspace(0, max_removal_fraction, 20):
        n_remove = int(removal_fraction * n_nodes)
        G_temp = G.copy()
        
        for _ in range(n_remove):
            if G_temp.number_of_nodes() > 0:
                # Find node with highest centrality
                centrality_temp = centrality_func(G_temp)
                if centrality_temp:
                    target_node = max(centrality_temp, key=centrality_temp.get)
                    G_temp.remove_node(target_node)
        
        # Find largest component
        if G_temp.number_of_nodes() > 0:
            largest_cc = max(nx.connected_components(G_temp), key=len)
            giant_component_size = len(largest_cc) / n_nodes
        else:
            giant_component_size = 0
        
        results.append({
            'removal_fraction': removal_fraction,
            'giant_component_size': giant_component_size
        })
    
    return pd.DataFrame(results)

# Compare different attack strategies
strategies = {
    'Degree': nx.degree_centrality,
    'Betweenness': nx.betweenness_centrality,
    'Closeness': nx.closeness_centrality
}

attack_results = {}
for name, func in strategies.items():
    attack_results[name] = analyze_targeted_attacks(G_scale_free, func)

print("Targeted Attack Analysis Complete!")

In [ ]:
# Plot comparison
plt.figure(figsize=(10, 6))
for name, results in attack_results.items():
    plt.plot(results['removal_fraction'], results['giant_component_size'], 
            'o-', label=name, linewidth=2)

plt.xlabel('Fraction of Nodes Removed')
plt.ylabel('Giant Component Size (Normalized)')
plt.title('Network Robustness: Targeted Attacks')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()